In [2]:
import sys
#sys.builtin_module_names
!{sys.executable} -m pip install numpy
#https://stackoverflow.com/questions/48754352/python-package-not-found-in-jupyter-even-after-running-pip-install?rq=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 10.6 MB/s eta 0:00:0000:0100:01


In [3]:
import random
import numpy as np

In [6]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.current_player = 1 # start with player 1
        self.moves = [] 
        self.p1_pits_index = [0, self.pits_per_player-1] 
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1] 
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

        self.winner = 0 # stores the winner for easy access later

    
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)

    
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        pit -= 1 #accounts for 1-indexing

        #populates valid_range with the indices of pits belonging to the current_player
        if self.current_player == 1:
            valid_range = range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)
        else:
            valid_range = range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)

        #checks whether the selected pit is within the range of pits belonging to the current_player
        if pit < len(valid_range):
            #creates a list with all the valid ranges (converted from range()) then grabs the value at index 'pit'
            # aka: converts the pit number to match the pit's index in the board
            pit_index = list(valid_range)[pit]
        else: 
            return False
        
        if self.board[pit_index] == 0: #checks if the pit is empty at the (determined to be valid) index
            return False

        return True

    
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        valid_pits = []
        if self.current_player == 1:
            start, end = self.p1_pits_index #unpacks p1's pit indices
        else:
            start, end = self.p2_pits_index #unpacks p2's pit indices

        for i in range(start, end + 1): #generates indices of current player's pits
            if self.board[i] > 0: #checks whether each pit is empty
                valid_pits.append(i - start + 1) #if not empty, pit index gets added to the list of valid indices

        if not valid_pits: #if this happens, there is an issue with the win calculation or with this generator
            return None
        
        return random.choice(valid_pits) #randomly chooses a pit (move) from the list of valid moves

    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        if not self.valid_move(pit): #checks if the move is valid (if not, player is not switched)
            print(f"INVALID MOVE: Pit {pit} by Player {self.current_player}")
            return self.board

        if self.winning_eval(): #checks to see if the game has already been won
            print("GAME OVER")
            return self.board

        pit -= 1 #accounts for 1-indexing

        #converts the pit number to match the pit's index in the board
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + pit
        else:
            pit_index = self.p2_pits_index[0] + pit

        stones = self.board[pit_index] #grabs the number of stones in the pit 
        self.board[pit_index] = 0 #empties the pit
        index = pit_index #initializes the iterable

        while stones > 0:
            index = (index + 1) % len(self.board) #if we're past the bounds of the board (which translates
            # to index equaling the lenth of the board), our index is set to the start of the list

            #ensures that you don't place stones in the other player's mancala
            if self.current_player == 1 and index == self.p2_mancala_index:
                continue
            if self.current_player == 2 and index == self.p1_mancala_index:
                continue

            self.board[index] += 1 #adds a stone to the current pit
            stones -= 1 #removes a stone from our 'hand'

        #implements the 'if a player's last stone lands in an empty pit on their side of the board, take that stone
        # and all the stones in the opposite pit (on the other player's side) and put them into your own mancala' rule
        if self.current_player == 1:
            #checks that the final pit is on player 1's side, and that the pit was empty (and now has 1 stone)
            if index in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1) and self.board[index] == 1:
                opposite_index = self.p2_pits_index[1] - (index - self.p1_pits_index[0])
                #calculates P2's index
                captured = self.board[opposite_index]
                #places the stones into P1's mancala
                self.board[self.p1_mancala_index] += captured + 1
                self.board[index] = 0
                self.board[opposite_index] = 0
        else:
            #checks that the final pit is on player 2's side, and that the pit was empty (and now has 1 stone)
            if index in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1) and self.board[index] == 1:
                opposite_index = self.p1_pits_index[1] - (index - self.p2_pits_index[0])
                #calculates P1's index
                captured = self.board[opposite_index]
                #places the stones into P2's mancala
                self.board[self.p2_mancala_index] += captured + 1
                self.board[index] = 0
                self.board[opposite_index] = 0

        #adds the move to 'history'
        self.moves.append((self.current_player, pit + 1))

        #implements the 'if your final stone was dropped into your mancala, take another turn, otherwise 
        # switch players' rule
        #if (self.current_player == 1 and index != self.p1_mancala_index) or (self.current_player == 2 and index != self.p2_mancala_index):
            #self.current_player = 2 if self.current_player == 1 else 1

        #skips the previously established rule, swaps players every turn
        self.current_player = 2 if self.current_player == 1 else 1
        
        return self.board

    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state (if either of the players' pits are all empty)
        """
        #checks each pit in each players' side of the board to see if they're empty
        p1_side_empty = all(stone == 0 for stone in self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
        p2_side_empty = all(stone == 0 for stone in self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])

        
        if p1_side_empty or p2_side_empty: # win condition met
            #adds the sum of all stones left within each players' pit to their respective mancalas
            self.board[self.p1_mancala_index] += sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
            self.board[self.p2_mancala_index] += sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])
            #empties each player's pits
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[i] = 0

            #checks who won (has more stones in mancala)
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                self.winner = 1 # winner is player 1
                print("Player 1 wins!")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                self.winner = 2 # winner is player 2
                print("Player 2 wins!")
            else:
                #print("It’s a tie!")
                pass
            return True
        return False

In [9]:
p1_wins = 0
p2_wins = 0
ties = 0
moves_to_end = [] #list of how many moves each game took to reach completion

num_games = 100

for match in range(num_games): #runs num_games times
    moves = 0 #number of moves this game took to win, initialized to 0

    game = Mancala(pits_per_player=6, stones_per_pit=4) #new game
    
    while not game.winning_eval():
        pit = game.random_move_generator() #each player chooses a move randomly
        
        if pit is None: 
            #error. pit being None means that, theoretically, all the player's pits were empty, 
            # which should have been caught by winning_eval()
            print("Error")
            break
        
        game.play(pit)
        moves += 1
    
    if game.winner == 1: #checks if player 1 won
        p1_wins += 1
    elif game.winner == 2: #checks if player 2 won
        p2_wins += 1
    else:
        ties += 1
    
    moves_to_end += [moves] 

print(f"p1 wins: {p1_wins}, {(p1_wins/num_games) * 100}%")
print(f"p2 wins: {p2_wins}, {(p2_wins/num_games) * 100}%")
print(f"Ties: {ties}, {(ties/num_games) * 100}%")
print("average moves: ", np.mean(moves_to_end))

        

Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
It’s a tie!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
It’s a tie!
It’s a tie!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
It’s a tie!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
It’s a tie!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 w